In [ ]:
import os
import glob
import pandas as pd
from tqdm import tqdm
import AMD_Tools3 as amd

%matplotlib inline

In [ ]:
simdata = [pd.read_csv(csvpath) for csvpath in glob.glob('simdata/*.csv')]
simdata = pd.concat(simdata, axis=0)

print('simdata.shape:', simdata.shape)
simdata.head()

In [ ]:
#
# DVI map
#

import folium

simdata_grouped = simdata.groupby(by=['meshcode', 'year']).DVI.max()
print('len(DVI<1.9):', (simdata_grouped < 1.9).sum())
print('len(simdata):', len(simdata_grouped))

def meshcode2lat(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[0]

def meshcode2lon(meshcode):
    latlon = amd.mesh2lalo(meshcode)
    return latlon[1]

simdata2 = simdata_grouped.reset_index()
simdata2['lat'] = simdata2.meshcode.astype(str).apply(meshcode2lat)
simdata2['lon'] = simdata2.meshcode.astype(str).apply(meshcode2lon)

fig, axes = plt.subplots(6, 7, figsize=(15, 10))
for year, ax in zip(simdata2.year.unique(), axes.flatten()):
    
    #scatter plot
    df = simdata2[simdata2.year == year].copy()
    df.plot.scatter(x='lon', y='lat', c='DVI', ax=ax)
    
    #scatter map plot
    # map = folium.Map(location=[35.170833333333334, 138.18125], zoom_start=6)
    # for i, row in df.iterrows():
    #     folium.CircleMarker([row['lat'], row['lon']], radius=1,
    #                 popup=str(row['DVI']), fill_color=row['DVI'], fill_opacity=0.5).add_to(map)
    # map
    # map.save(f'output/heatmap_{year}.html')
    # break
    
#show scatter
plt.tight_layout()
plt.show()

In [ ]:
#
# Histogram of inputs
#

dt = ['DVI', 'DVR', 'TAV', 'TMX', 'RAD', 'DL', 'PPM']
fig, axes = plt.subplots(2, 4, figsize=(12, 5))
cnt = 0

for _dt in tqdm(dt):
    ax = axes.flatten()[cnt]
    df = simdata[_dt]
    df.plot.hist(ax=ax)
    ax.set_xlabel(_dt)
    ax.set_ylabel('Frequency')
    cnt += 1
    
fig.tight_layout()

In [ ]:
#
# Scatter matrix of inputs
#

import seaborn as sns
sns.set(style="ticks")

dt = ['DVI', 'DVR', 'TAV', 'TMX', 'RAD', 'DL', 'PPM']
df = simdata[dt]
df = df.sample(frac=.005, axis=0)  # random sampling to make it fast
print('df.shape:', df.shape)

sns.pairplot(df, size=1.)